In [12]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-
"""
Created on Wed Jul 22 10:42:54 2020
@ date; 31 august 2021
@author: Claudia Acquistapace
@goal: plot radiosonde quantities for the case study of the 2nd february using data from l'atalante
"""

# importing necessary libraries
import matplotlib.pyplot as plt
from matplotlib import rcParams
import matplotlib
import numpy as np
import xarray as xr
from datetime import datetime
import matplotlib.dates as mdates
import glob
from myFunctions import lcl
from myFunctions import f_closest
from warnings import warn
import numpy as np
import pandas as pd
import datetime as dt
import matplotlib.pyplot as plt
from scipy import interpolate
import custom_color_palette as ccp
from matplotlib import rcParams
import matplotlib.ticker as ticker

# reading radiosondes
path_RS = '/Volumes/Extreme SSD/work/006_projects/001_Prec_Trade_Cycle/radiosondes_atalante/case_1/'
file_list_RS = [path_RS+'EUREC4A_Atalante_Vaisala-RS_L2_v3.0.0.nc', path_RS+'EUREC4A_Atalante_Meteomodem-RS_L2_v3.0.0.nc', path_RS+'EUREC4A_MS-Merian_Vaisala-RS_L2_v3.0.0.nc']
path_out = '/Volumes/Extreme SSD/work/006_projects/001_Prec_Trade_Cycle/post_processed_data/'
# +
# read radiosonde from vaisala
data_RS_vaisala = xr.open_dataset(file_list_RS[0])

# read time of the launch for all radiosondes
time_launch = pd.to_datetime(data_RS_vaisala.launch_time.values)
sounding_id = data_RS_vaisala.sounding.values

# find indeces for the cold patch
ind = np.where((time_launch > datetime(2020,2,2,0,0,0)) * (time_launch < datetime(2020,2,3,23,59,59)))[0]
start = sounding_id[ind[0]]
end = sounding_id[ind[-1]]

# selecting data_RS
data_vaisala_sel = data_RS_vaisala.sel(sounding=slice(start, end))
print('n rs vaisala', len(data_vaisala_sel))

# +
# read radiosonde from meteogram
data_RS_meteo = xr.open_dataset(file_list_RS[1])

# read time of the launch for all radiosondes
time_launch = pd.to_datetime(data_RS_meteo.launch_time.values)
sounding_id = data_RS_meteo.sounding.values

# find indeces for the cold patch
ind = np.where((time_launch > datetime(2020,2,2,0,0,0)) * (time_launch < datetime(2020,2,3,23,59,59)))[0]
start = sounding_id[ind[0]]
end = sounding_id[ind[-1]]

# selecting data_RS
data_meteo_sel = data_RS_meteo.sel(sounding=slice(start, end))
print('n rs meteomodem', len(data_meteo_sel))

# reading RS merian
data_RS_vaisala_MM = xr.open_dataset(file_list_RS[2])

# read time of the launch for all radiosondes
time_launch_mm = pd.to_datetime(data_RS_vaisala_MM.launch_time.values)
sounding_id_mm = data_RS_vaisala_MM.sounding.values

# find indeces for the cold patch
ind_mm = np.where((time_launch_mm > datetime(2020,2,2,0,0,0)) * (time_launch_mm < datetime(2020,2,3,23,59,59)))[0]
start_mm = sounding_id_mm[ind_mm[0]]
end_mm = sounding_id_mm[ind_mm[-1]]

# selecting data_RS
data_vaisala_mm_sel = data_RS_vaisala_MM.sel(sounding=slice(start_mm, end_mm))
print('n rs vaisala mm', len(data_vaisala_mm_sel))

# +
# merging the two selected datasets
data_patch = xr.merge([data_vaisala_sel, data_meteo_sel, data_vaisala_mm_sel])
print()

# re-ordering files in temporal order
sounding_id = pd.to_datetime(data_patch.launch_time.values)

# re-ordering and saving order of indeces in ind_sorted
soundings_strings_sorted = np.sort(sounding_id)
ind_sorted = np.argsort(sounding_id)


n rs vaisala 19
n rs meteomodem 19
n rs vaisala mm 19



/Users/claudia/opt/anaconda3/lib/python3.8/site-packages/xarray/core/alignment.py:307: FutureWarning: Index.__or__ operating as a set operation is deprecated, in the future this will be a logical operation matching Series.__or__.  Use index.union(other) instead
  index = joiner(matching_indexes)


In [13]:
soundings_strings_sorted

array(['2020-02-02T00:44:19.583056000', '2020-02-02T02:11:00.780326000',
       '2020-02-02T04:40:49.757005000', '2020-02-02T06:00:07.164235000',
       '2020-02-02T08:41:12.869950000', '2020-02-02T10:13:51.456500000',
       '2020-02-02T12:04:55.877013000', '2020-02-02T12:44:30.460057000',
       '2020-02-02T13:22:52.895287000', '2020-02-02T14:14:09.173317000',
       '2020-02-02T16:45:18.436984000', '2020-02-02T18:13:03.904394000',
       '2020-02-02T19:32:36.000000000', '2020-02-02T20:15:04.000000000',
       '2020-02-02T21:00:07.000000000', '2020-02-02T21:49:56.000000000',
       '2020-02-02T22:40:34.000000000', '2020-02-02T22:46:37.763015000',
       '2020-02-02T23:36:42.000000000', '2020-02-03T00:03:06.060986000',
       '2020-02-03T00:29:08.000000000', '2020-02-03T01:21:42.000000000',
       '2020-02-03T02:04:00.000000000', '2020-02-03T02:41:40.043016000',
       '2020-02-03T02:54:45.000000000', '2020-02-03T03:46:02.000000000',
       '2020-02-03T04:18:43.510016000', '2020-02-03

In [14]:
len(soundings_strings_sorted)

53

In [3]:
len(soundings_strings_sorted)

51